In [14]:
import os
import numpy as np
import pandas as pd
from scipy.stats import mannwhitneyu

# --------------------------------------------------
# File paths (adjust if needed)
# --------------------------------------------------
FYE_path = "/content/sample_data/UNO_Student_Experience_Survey 1 (1).csv"
NFYE_path = "/content/sample_data/UNO_Student_Experience_Survey_2.csv"


# --------------------------------------------------
# Load and clean Qualtrics CSV
# --------------------------------------------------
def load_csv(path, group_label, min_progress=0):
    df = pd.read_csv(path)

    # Drop first two metadata rows
    df = df.iloc[2:, :].reset_index(drop=True)

    # Convert numeric columns
    if "Progress" in df.columns:
        df["Progress"] = pd.to_numeric(df["Progress"], errors="coerce")
    if "Duration (in seconds)" in df.columns:
        df["Duration (in seconds)"] = pd.to_numeric(
            df["Duration (in seconds)"], errors="coerce"
        )

    # Keep finished responses only
    if "Finished" in df.columns:
        df = df[df["Finished"] == "True"]

    # Apply progress filter
    if "Progress" in df.columns and min_progress is not None:
        df = df[df["Progress"] >= min_progress]

    df["group"] = group_label
    return df


# --------------------------------------------------
# Recode Likert scales and other question types
# --------------------------------------------------
def recode_scales(df):
    agree_map = {
        "Strongly disagree.": 1,
        "Disagree.": 2,
        "Neutral.": 3,
        "Agree.": 4,
        "Strongly agree.": 5
    }
    agree_set = set(agree_map.keys())

    connected_map = {
        "Not connected at all.": 1,
        "Neutral.": 2,
        "Somewhat connected.": 3,
        "Connected.": 4
    }
    connected_set = set(connected_map.keys())

    freq_map = {"0": 0, "1": 1, "2": 2, "3": 3, "4+": 4}
    freq_set = set(freq_map.keys())

    yesno_map = {
        "No.": 0, "Yes.": 1,
        "No": 0, "Yes": 1,
        "Prefer not to say": np.nan,
        "Prefer not to say.": np.nan
    }
    yesno_set = {k for k, v in yesno_map.items() if v in (0, 1)}

    gpa_map = {
        "2.5 or lower": 1, "2.5-3.0": 2, "3.0-3.25": 3,
        "3.25-3.5": 4, "3.5-3.75": 5, "3.75+": 6
    }
    gpa_set = set(gpa_map.keys())

    hours_map = {
        "5 or fewer hours.": 1, "6 - 10 hours.": 2, "11 - 15 hours.": 3,
        "16 - 20 hours.": 4, "21 - 25 hours.": 5, "Greater than 25 hours.": 6
    }
    hours_set = set(hours_map.keys())

    change_map = {
        "My grades have improved on average.": 3,
        "My grades have stayed the same on average.": 2,
        "My grades have worsened on average.": 1,
        "My time management skills have improved.": 3,
        "My time management skills have stayed the same.": 2,
        "My time management skills have worsened.": 1,
        "I study more than before.": 3,
        "My study habits have not changed.": 2,
        "I study less than before.": 1
    }
    change_set = set(change_map.keys())

    for col in df.columns:
        if not col.startswith("Q"):
            continue

        vals = set(df[col].dropna().unique())
        if not vals:
            continue

        if vals.issubset(agree_set):
            df[col + "_num"] = df[col].map(agree_map).astype(float)
        elif vals.issubset(connected_set):
            df[col + "_num"] = df[col].map(connected_map).astype(float)
        elif vals.issubset(freq_set):
            df[col + "_num"] = df[col].map(freq_map).astype(float)
        elif vals.issubset(yesno_set):
            df[col + "_num"] = df[col].map(yesno_map).astype(float)
        elif vals.issubset(gpa_set):
            df[col + "_num"] = df[col].map(gpa_map).astype(float)
        elif vals.issubset(hours_set):
            df[col + "_num"] = df[col].map(hours_map).astype(float)
        elif vals.issubset(change_set):
            df[col + "_num"] = df[col].map(change_map).astype(float)

    return df


# --------------------------------------------------
# Score financial literacy (Q37–Q41)
# --------------------------------------------------
def score_financial_literacy(df):
    def score(series, correct):
        return np.where(
            series.isna(), np.nan,
            np.where(series == correct, 1, 0)
        )

    if "Q37" in df.columns:
        df["Q37_correct"] = score(df["Q37"], "More than $102.")
    if "Q38" in df.columns:
        df["Q38_correct"] = score(df["Q38"], "Less than today.")
    if "Q39" in df.columns:
        df["Q39_correct"] = score(df["Q39"], "They will fall.")
    if "Q40" in df.columns:
        df["Q40_correct"] = score(df["Q40"], "True.")
    if "Q41" in df.columns:
        df["Q41_correct"] = score(df["Q41"], "False.")

    fin_cols = [c for c in df.columns if c.endswith("_correct")]
    if fin_cols:
        df["FinLit_Total"] = df[fin_cols].sum(axis=1)

    return df


# --------------------------------------------------
# MASTER FUNCTION – combine FYE and NonFYE
# --------------------------------------------------
def prepare_analysis_dataset():
    pass


In [15]:
import os
import numpy as np
import pandas as pd
from scipy.stats import mannwhitneyu

# --------------------------------------------------
# File paths (adjust if needed)
# --------------------------------------------------
FYE_path = "/content/sample_data/UNO_Student_Experience_Survey 1 (1).csv"
NFYE_path = "/content/sample_data/UNO_Student_Experience_Survey_2.csv"


# --------------------------------------------------
# Load and clean Qualtrics CSV
# --------------------------------------------------
def load_csv(path, group_label, min_progress=0):
    df = pd.read_csv(path)

    # Drop first two metadata rows
    df = df.iloc[2:, :].reset_index(drop=True)

    # Convert numeric columns
    if "Progress" in df.columns:
        df["Progress"] = pd.to_numeric(df["Progress"], errors="coerce")
    if "Duration (in seconds)" in df.columns:
        df["Duration (in seconds)"] = pd.to_numeric(
            df["Duration (in seconds)"], errors="coerce"
        )

    # Keep finished responses only
    if "Finished" in df.columns:
        df = df[df["Finished"] == "True"]

    # Apply progress filter
    if "Progress" in df.columns and min_progress is not None:
        df = df[df["Progress"] >= min_progress]

    df["group"] = group_label
    return df


# --------------------------------------------------
# Recode Likert scales and other question types
# --------------------------------------------------
def recode_scales(df):
    agree_map = {
        "Strongly disagree.": 1,
        "Disagree.": 2,
        "Neutral.": 3,
        "Agree.": 4,
        "Strongly agree.": 5
    }
    agree_set = set(agree_map.keys())

    connected_map = {
        "Not connected at all.": 1,
        "Neutral.": 2,
        "Somewhat connected.": 3,
        "Connected.": 4
    }
    connected_set = set(connected_map.keys())

    freq_map = {"0": 0, "1": 1, "2": 2, "3": 3, "4+": 4}
    freq_set = set(freq_map.keys())

    yesno_map = {
        "No.": 0, "Yes.": 1,
        "No": 0, "Yes": 1,
        "Prefer not to say": np.nan,
        "Prefer not to say.": np.nan
    }
    yesno_set = {k for k, v in yesno_map.items() if v in (0, 1)}

    gpa_map = {
        "2.5 or lower": 1, "2.5-3.0": 2, "3.0-3.25": 3,
        "3.25-3.5": 4, "3.5-3.75": 5, "3.75+": 6
    }
    gpa_set = set(gpa_map.keys())

    hours_map = {
        "5 or fewer hours.": 1, "6 - 10 hours.": 2, "11 - 15 hours.": 3,
        "16 - 20 hours.": 4, "21 - 25 hours.": 5, "Greater than 25 hours.": 6
    }
    hours_set = set(hours_map.keys())

    change_map = {
        "My grades have improved on average.": 3,
        "My grades have stayed the same on average.": 2,
        "My grades have worsened on average.": 1,
        "My time management skills have improved.": 3,
        "My time management skills have stayed the same.": 2,
        "My time management skills have worsened.": 1,
        "I study more than before.": 3,
        "My study habits have not changed.": 2,
        "I study less than before.": 1
    }
    change_set = set(change_map.keys())

    for col in df.columns:
        if not col.startswith("Q"):
            continue

        vals = set(df[col].dropna().unique())
        if not vals:
            continue

        if vals.issubset(agree_set):
            df[col + "_num"] = df[col].map(agree_map).astype(float)
        elif vals.issubset(connected_set):
            df[col + "_num"] = df[col].map(connected_map).astype(float)
        elif vals.issubset(freq_set):
            df[col + "_num"] = df[col].map(freq_map).astype(float)
        elif vals.issubset(yesno_set):
            df[col + "_num"] = df[col].map(yesno_map).astype(float)
        elif vals.issubset(gpa_set):
            df[col + "_num"] = df[col].map(gpa_map).astype(float)
        elif vals.issubset(hours_set):
            df[col + "_num"] = df[col].map(hours_map).astype(float)
        elif vals.issubset(change_set):
            df[col + "_num"] = df[col].map(change_map).astype(float)

    return df


# --------------------------------------------------
# Score financial literacy (Q37–Q41)
# --------------------------------------------------
def score_financial_literacy(df):
    def score(series, correct):
        return np.where(
            series.isna(), np.nan,
            np.where(series == correct, 1, 0)
        )

    if "Q37" in df.columns:
        df["Q37_correct"] = score(df["Q37"], "More than $102.")
    if "Q38" in df.columns:
        df["Q38_correct"] = score(df["Q38"], "Less than today.")
    if "Q39" in df.columns:
        df["Q39_correct"] = score(df["Q39"], "They will fall.")
    if "Q40" in df.columns:
        df["Q40_correct"] = score(df["Q40"], "True.")
    if "Q41" in df.columns:
        df["Q41_correct"] = score(df["Q41"], "False.")

    fin_cols = [c for c in df.columns if c.endswith("_correct")]
    if fin_cols:
        df["FinLit_Total"] = df[fin_cols].sum(axis=1)

    return df


# --------------------------------------------------
# MASTER FUNCTION – combine FYE and NonFYE
# --------------------------------------------------
def prepare_analysis_dataset(FYE_path, NFYE_path, min_progress=0):
    fye = load_csv(FYE_path, group_label="FYE", min_progress=min_progress)
    nfye = load_csv(NFYE_path, group_label="NonFYE", min_progress=min_progress)

    # Align columns
    common_cols = fye.columns.intersection(nfye.columns)
    fye = fye[common_cols].copy()
    nfye = nfye[common_cols].copy()

    # Recode & score
    fye = recode_scales(fye)
    nfye = recode_scales(nfye)

    fye = score_financial_literacy(fye)
    nfye = score_financial_literacy(nfye)

    combined = pd.concat([fye, nfye], ignore_index=True)
    return combined


# --------------------------------------------------
# Run Mann–Whitney U tests on all *_num variables
# --------------------------------------------------
def run_mann_whitney_tests(df, group_col="group"):
    results = []

    num_cols = [col for col in df.columns if col.endswith("_num")]

    for col in num_cols:
        group1 = df[df[group_col] == "FYE"][col].dropna()
        group2 = df[df[group_col] == "NonFYE"][col].dropna()

        if len(group1) > 0 and len(group2) > 0:
            stat, p = mannwhitneyu(group1, group2, alternative="two-sided")
            results.append({
                "Variable": col,
                "FYE_n": len(group1),
                "NonFYE_n": len(group2),
                "U_statistic": stat,
                "p_value": p
            })

    return pd.DataFrame(results)


# --------------------------------------------------
# MAIN EXECUTION
# --------------------------------------------------
combined = prepare_analysis_dataset(FYE_path, NFYE_path, min_progress=0)
print("Combined shape:", combined.shape)

mannwhitney_results = run_mann_whitney_tests(combined)
mannwhitney_results_sorted = mannwhitney_results.sort_values("p_value")
mannwhitney_results_sorted


Combined shape: (11, 90)


,Variable,FYE_n,NonFYE_n,U_statistic,p_value
16,Q21_num,1,9,9.0,0.085192
12,Q12_num,1,9,1.0,0.190430
3,Q14_num,1,9,8.5,0.201243
23,Q34_num,1,9,1.0,0.273322
18,Q23_num,1,9,8.0,0.274916
17,Q22_num,1,9,8.0,0.281198
5,Q30_num,1,9,7.5,0.335924
9,Q17_num,1,9,1.5,0.361310
15,Q20_num,1,9,7.0,0.441418
20,Q25_num,1,9,7.0,0.454486
